In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import re
from itertools import product
from tqdm import tqdm
import datetime

In [6]:
def getRaceData(race_id): 
    url = f"https://db.netkeiba.com/race/{race_id}/"
    driver.get(url)
    
    assert driver.title[0] != '｜'
    
    html = driver.page_source

    df = pd.read_html(html)
    race_result = df[0]
    race_result['race_id'] = race_id

    header_text = driver.find_element_by_class_name('data_intro').text
    header_text = header_text.split('\n')
    race_title = header_text[1]
    # 過去の〜〜　リンクの有無によって変わるので
    race_info_index = 4 if header_text[3][0:2] == '過去' else 3
    course_info = header_text[2].split('/')
    course_type = course_info[0][0]
    course_direction = course_info[0][1]
    course_distance = course_info[0][2:6]
    weather = course_info[1].replace(' ', '').replace('：', ':').replace('天候:', '')
    condition = course_info[2].replace(' ', '').replace('：', ':').replace('芝:', '').replace('ダート:', '')
    race_class = header_text[race_info_index].split(' ')[2]
    race_date = header_text[race_info_index].split(' ')[0]
    course_name = re.search('回(.+)\d+.*', header_text[race_info_index].split(' ')[1])[1]

    race_info_data=[race_date,race_title,race_class, course_name, course_type, course_direction, course_distance, condition, weather]
    
    return (race_result, race_info_data)

In [7]:
#Chromeを操作
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(executable_path="/Users/k-yasumoto/downloads/chromedriver", chrome_options=options)

<ipython-input-7-e03bfc11aade>:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path="/Users/k-yasumoto/downloads/chromedriver", chrome_options=options)


In [6]:
race_result_list = []
error_list = []

race_info_columns=['race_date','race_title', 'race_class','course_name','course_type', 'course_direction', 'course_distance', 'condition','weather']
df_race_info = pd.DataFrame(columns=race_info_columns)
df_race_info.index.name='race_id'

for year in range(2010, 2021):
    print('start ', year)
    year = str(year)
    for place in tqdm(range(1,11)):
        place = str(place).zfill(2)
        for time in range(1,7):
            time = str(time).zfill(2)
            date = 1
            flg = True
            while flg:
                for race in range(1,13):
                    race = str(race).zfill(2)
                    race_id = year + place + time + str(date).zfill(2) + race
                    if int(race_id) < 201910021212:
                        continue
                    try:
                        race_result, race_info_data = getRaceData(race_id)
                        df_race_info.loc[race_id]=race_info_data
                        race_result_list.append(race_result)
                    except AssertionError: # 変える
                        error_list.append(race_id)
                        flg = False
                        break
                date += 1
                if date > 15:
                    flg = False
    print()

  0%|          | 0/10 [00:00<?, ?it/s]

start  2010

start  2011

start  2012

start  2013

start  2014

start  2015

start  2016

start  2017

start  2018

start  2019


  0%|          | 0/10 [00:00<?, ?it/s]


start  2020


100%|██████████| 10/10 [33:10<00:00, 199.07s/it]

In [15]:
date_str = datetime.datetime.now().strftime("%Y%m%d")
pd.concat(race_result_list, ignore_index=True).to_csv("../../data/scraped/result/race_result_" + date_str + ".csv")
df_race_info.to_csv("../../data/scraped/result/race_info_" + date_str + ".csv")

In [10]:
result = getRaceData(202008030911)

In [11]:
result

(    着順  枠番  馬番         馬名  性齢  斤量    騎手     タイム     着差 ﾀｲﾑ指数  ...     単勝  人気  \
 0    1   4   5   オメガパフューム  牡5  59  北村友一  1:56.0    NaN    **  ...    5.3   3   
 1    2   7  11    ヴェンジェンス  牡7  57   幸英明  1:56.2  1.1/4    **  ...    8.6   5   
 2    3   5   7   ゴールドドリーム  牡7  58  藤岡佑介  1:56.5      2    **  ...    4.3   2   
 3    4   3   3  ヒストリーメイカー  牡6  56  畑端省吾  1:56.8  1.1/2    **  ...   64.2  10   
 4    5   6  10   スワーヴアラミス  牡5  57  松田大作  1:56.9    1/2    **  ...    5.4   4   
 5    6   7  12    マグナレガーロ  牡5  56  北村宏司  1:57.0    1/2    **  ...   12.0   6   
 6    7   8  14    ダンツゴウユウ  牡6  56   酒井学  1:57.0     ハナ    **  ...   59.7   9   
 7    8   1   1     アシャカトブ  牡4  56   武藤雅  1:57.2  1.1/2    **  ...   58.5   8   
 8    9   4   6     ヒロブレイブ  牡7  56  国分恭介  1:57.3     クビ    **  ...  364.1  14   
 9   10   2   2    ロードレガリス  牡5  56  池添謙一  1:57.4    1/2    **  ...    3.0   1   
 10  11   6   9        ミツバ  牡8  58  松若風馬  1:57.7  1.3/4    **  ...  176.4  13   
 11  12   3   4      ハヤヤッコ  